## 빗썸 API 실습

In [ ]:
import os
import hashlib
import math
import time
import base64
import hmac, hashlib
from urllib.parse import urlencode
import requests

### 환경변수 호출

In [ ]:
from dotenv import load_dotenv

load_dotenv()

ACCESS_KEY = os.environ["BITHUMB_API_ACCESS_KEY"]
SECRET_KEY = os.environ["BITHUMB_API_SECRET_KEY"]
SERVER_URL = os.environ["BITHUMB_API_SERVER_URL"]
SERVER_URL

os.environ을 조회 해 보았을때 .env에서 추가한 변수들이 정상적으로 추가되어 있는지 확인해보자  
만약 정상적으로 로딩이 안된다면 아래 명령어로 .env 파일을 로딩해보자  
`%dotenv {.env파일의 절대 또는 상대경로}/.env`

In [ ]:
os.environ

### 토큰 만들기

In [ ]:
def timestamp_millisec():
    mt_string = "%f %d" % math.modf(time.time())
    mt_array = mt_string.split(" ")[:2]
    return mt_array[1] + mt_array[0][2:5]
    
def create_token(endpoint, rgParams):
    """Api-Sign and Api-Nonce information generation.

    # - nonce: it is an arbitrary number that may only be used once.
    # - api_sign: API signature information created in various combinations values.
    """
    endpoint_item_array = {"endpoint": endpoint}

    uri_array = dict(endpoint_item_array, **rgParams)  # Concatenate the two arrays.
    print(f"uri_array {uri_array}")

    str_data = urlencode(uri_array)
    nonce = timestamp_millisec()

    data = endpoint + chr(0) + str_data + chr(0) + nonce
    utf8_data = data.encode("utf-8")

    key = SECRET_KEY
    utf8_key = key.encode("utf-8")

    h = hmac.new(bytes(utf8_key), utf8_data, hashlib.sha512)
    hex_output = h.hexdigest()
    utf8_hex_output = hex_output.encode("utf-8")
    api_sign = base64.b64encode(utf8_hex_output)
    utf8_api_sign = api_sign.decode("utf-8")

    url = SERVER_URL + endpoint
    headers = {
        "Api-Key": ACCESS_KEY,
        "Api-Sign": utf8_api_sign,
        "Api-Nonce": nonce,
        "Content-Type": "application/x-www-form-urlencoded",
    }
    
    return (url, headers, str_data)

### 계좌 조회 토큰 생성

In [ ]:
account_query = {
    "order_currency": "BTC",
}
create_token("/info/account", account_query)

### 계좌 조회하기

In [ ]:
def query_account(query):
    print(query)
    info = create_token("/info/account", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

In [ ]:
query_account(account_query)

### 계좌 잔고 조회하기

In [ ]:
def query_account_balance(query):
    print(query)
    info = create_token("/info/balance", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

In [ ]:
query_account_balance({})

### 주문 토큰 생성

In [ ]:
query = {
    "order_currency": "BTC",
    "payment_currency": "KRW",
    "type": "bid",
    "units": str(0.0001),
    "price": str(30000000),
}
create_token("/trade/place", query)

### 주문하기

In [ ]:
def send_order(query):
    print(query)
    info = create_token("/trade/place", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

In [ ]:
send_order(query)

### 대기 주문 조회 토큰 생성

In [ ]:
orders_query = {
    "order_currency": "BTC",
}
create_token("/info/orders", orders_query)

### 대기 주문 조회

In [ ]:
def query_waiting_order(query):
    info = create_token("/info/orders", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

In [ ]:
query_waiting_order(orders_query)

### 개별 주문 조회 토큰 생성

In [ ]:
order_query = {
    "order_currency": "BTC",
    "order_id": "C0101000000426680316"
}
create_token("/info/order_detail", order_query)

### 개별 주문 조회

In [ ]:
def query_order(query):
    info = create_token("/info/order_detail", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

In [ ]:
query_order(order_query)

### 주문 취소

In [ ]:
cancel_query = {
    "order_currency": "BTC",
    "order_id": "C0101000000426680316",
    "type": "bid",
    "payment_currency": "KRW"
}
create_token("/trade/cancel", cancel_query)

In [ ]:
def cancel_order(query):
    info = create_token("/trade/cancel", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

In [ ]:
cancel_order(cancel_query)

### 최근 거래 내역 조회

In [ ]:
def get_trade_tick():
    querystring = {"count": "1"}

    response = requests.get(
        SERVER_URL + f"/public/transaction_history/BTC_KRW", params=querystring
    )
    return response.json()

In [ ]:
get_trade_tick()